# Notebook 02: Advanced Feature Engineering & Signal Audit

**Author:** Hector Carbajal  
**Version:** 1.1  
**Last Updated:** 2026-02

---

## Purpose
This notebook enriches the firmware telemetry with advanced categorical and temporal features. We perform a statistical signal audit to ensure our engineered features (Code Churn, Device Age, Lagged Errors) provide meaningful predictive power for the regression and classification models.

## Inputs
- `data/Processed/synthetic_firmware_features.csv` - Ground truth features from synthesis
- `data/Processed/error_rates.csv` - SQL-engineered error metrics
- `data/Processed/rma_by_firmware.csv` - SQL-engineered RMA metrics

## Key Engineered Features
- **Code Churn Intensity**: `lines_changed / days_since_release`
- **Fleet Maturity**: `avg_device_age_days` at time of patch release
- **Regression Lag**: `previous_version_error_rate`

## Table of Contents
1. [Setup & Data Ingestion](#setup)
2. [Feature Synthesis & Merging](#merging)
3. [🔍 Predictive Signal Audit (Correlation)](#signal)
4. [💡 Feature Engineering Insights](#findings)

In [1]:
# Setup
import pandas as pd
import numpy as np
import plotly.express as px
import sqlite3
import sys
from pathlib import Path

# Project imports
current_dir = Path.cwd()
project_root = current_dir.parents[0] if current_dir.name == "notebooks" else current_dir
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.config import DB_PATH, PROCESSED_DATA_DIR

# Aesthetic Config
px.defaults.template = "plotly_white"

conn = sqlite3.connect(DB_PATH)
print("✅ Setup complete. Connected to telemetry database.")

✅ Setup complete. Connected to telemetry database.


<a id="merging"></a>
## 1. Feature Synthesis & Merging
*Combining ground truth synthetic features with SQL-extracted signals.*

In [2]:
# Load ground truth synthetic features (1000 rows)
synthetic_features = pd.read_csv(PROCESSED_DATA_DIR / "synthetic_firmware_features.csv")

# Load SQL-engineered features (from notebook 01)
error_rates = pd.read_csv(PROCESSED_DATA_DIR / "error_rates.csv")
rma_fw = pd.read_csv(PROCESSED_DATA_DIR / "rma_by_firmware.csv")

# Merge SQL features onto synthetic truth using LEFT join to preserve all versions
features = synthetic_features.merge(error_rates, on="firmware_version", how="left", suffixes=('', '_sql'))
features = features.merge(rma_fw, on="firmware_version", how="left", suffixes=('', '_sql'))

# Clean up duplicate columns if any and handle NaNs from SQL join
# (If SQL didn't find errors for a version, it means 0 errors were recorded)
features['error_events'] = features['error_events'].fillna(0)
features['error_rate_per_10k'] = features['error_rate_per_10k'].fillna(0)
features['rmas'] = features['rmas'].fillna(0)
features['tickets'] = features['tickets'].fillna(0)
features['rma_rate'] = features['rma_rate'].fillna(0)

# Save full ML dataset
features.to_csv(PROCESSED_DATA_DIR / "firmware_features_ml.csv", index=False)
print(f"✅ Merged feature set saved: {len(features)} rows")

✅ Merged feature set saved: 1000 rows


<a id="signal"></a>
## 2. 🔍 Predictive Signal Audit
*Validating feature utility via Spearman Correlation (Non-linear relationship detection).*

In [ ]:
# Load combined features
features_df = pd.read_csv(PROCESSED_DATA_DIR / "firmware_features_ml.csv")

# IMPORTANT: For ML prediction, we can only use PRE-DEPLOYMENT features
# Post-deployment metrics (error_events, post_errors, tickets, rmas) would be data leakage
PRE_DEPLOYMENT_FEATURES = [
    'code_churn_score',
    'previous_version_error_rate', 
    'avg_device_age_days',
    'is_hotfix',
    'patch_security',
    'days_since_previous_release',
    'patch_size_mb',
    'files_changed',
    'lines_changed',
    'pre_errors'  # Errors from PREVIOUS version (known before deployment)
]

# Filter to only pre-deployment features + target
analysis_cols = [c for c in PRE_DEPLOYMENT_FEATURES if c in features_df.columns] + ['error_rate_per_10k']
analysis_df = features_df[analysis_cols]

# Calculate correlation with target
corr_matrix = analysis_df.select_dtypes(include=[np.number]).corr(method='spearman')
target_corr = corr_matrix['error_rate_per_10k'].sort_values(ascending=False).reset_index()
target_corr = target_corr[target_corr['index'] != 'error_rate_per_10k']

fig = px.bar(
    target_corr, 
    x='index', y='error_rate_per_10k', 
    title='Pre-Deployment Feature Correlation with Error Rate (Spearman)',
    labels={'error_rate_per_10k': 'Correlation Coefficient', 'index': 'Feature (Available Before Deployment)'},
    color='error_rate_per_10k', color_continuous_scale='RdBu_r'
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()

print("\\n📋 NOTE: Only PRE-DEPLOYMENT features are shown above.")
print("   Post-deployment metrics (error_events, tickets, rmas) are excluded to prevent data leakage.")

<a id="findings"></a>
## 💡 Feature Engineering Insights
**Operational Correlation Summary**

In [ ]:
# Get top pre-deployment feature (excluding target)
top_feature = target_corr.iloc[0]['index']
score = target_corr.iloc[0]['error_rate_per_10k']

print("="*80)
print("🚀 FEATURE UTILITY REPORT (Pre-Deployment Features Only)")
print("="*80)
print(f"1. PRIMARY SIGNAL: '{top_feature}' is the strongest pre-deployment predictor (ρ={score:.2f}).")
print(f"   This feature is available BEFORE firmware release, making it valid for predictive modeling.")
print(f"\n2. LAGGED IMPACT: 'previous_version_error_rate' captures historical risk persistence,")
print(f"   allowing the model to learn from past version performance.")
print(f"\n3. MULTICOLLINEARITY: High overlap detected between 'lines_changed' and 'patch_size_mb';")
print(f"   the normalized 'code_churn_score' is used as the primary complexity metric.")
print(f"\n4. DATA LEAKAGE PREVENTION: Post-deployment metrics (error_events, tickets, rmas)")
print(f"   are EXCLUDED from model training to ensure real-world applicability.")
print("="*80)